# Pymaceuticals Inc.
---

### Analysis

- Add your analysis here.
 

In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import scipy.stats as st

# Study data files
mouse_metadata_path = "../data/Mouse_metadata.csv"
study_results_path = "../data/Study_results.csv"

# Read the mouse data and the study results
mouse_metadata = pd.read_csv(mouse_metadata_path)
study_results = pd.read_csv(study_results_path)

# Combine the data into a single DataFrame
combined_data = pd.merge(mouse_metadata, study_results, on="Mouse ID")

# Display the data table for preview
combined_data.head()

In [ ]:
num_mice= len(combined_data["Mouse ID"].unique())
print(num_mice)

In [ ]:
# Our data should be uniquely identified by Mouse ID and Timepoint
# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint. 


In [ ]:
# Our data should be uniquely identified by Mouse ID and Timepoint

# Get the duplicate mice by ID number that shows up for Mouse ID and Timepoint
dupes_data = combined_data[combined_data.duplicated(["Mouse ID","Timepoint"])]

# duplicate mice by ID number
dupes_mice = dupes_data["Mouse ID"].unique()

print(dupes_mice,)


In [ ]:
# Optional: Get all the data for the duplicate mouse ID. 
dupes_data_mouse_id = combined_data.loc[combined_data["Mouse ID"]=="g989",:]

pd.DataFrame(dupes_data_mouse_id)


In [ ]:
# Create a clean DataFrame by dropping the duplicate mouse by its ID.
combined_data_clean = combined_data.drop(combined_data[combined_data["Mouse ID"]=="g989"].index)

# disply dataframe
combined_data_clean.head()

In [ ]:
# Number of mice IDs after dropped ID. Should = num_mice-1(248)
num_mice_clean= len(combined_data_clean["Mouse ID"].unique())
print(num_mice_clean)

## Summary Statistics

In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen

# Use groupby and summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume. 
# Assemble the resulting series into a single summary DataFrame.


In [ ]:
print(combined_data_clean.columns)


In [ ]:
# Generate a summary statistics table of mean, median, variance, standard deviation, and SEM of the tumor volume for each regimen
import numpy as np

# Group data by Drug Regimen
gd_DR= combined_data_clean.groupby('Drug Regimen')['Tumor Volume (mm3)']

# summary statistical methods to calculate the following properties of each drug regimen: 
# mean, median, variance, standard deviation, and SEM of the tumor volume.
gd_mean= gd_DR.mean()
gd_median = gd_DR.median()
gd_variance = gd_DR.var()
gd_std = gd_DR.std()
gd_sem = gd_DR.apply(lambda x: np.std(x)/ np.sqrt(len(x)))

# Assemble the resulting series into a single summary DataFrame.
ss_df= pd.DataFrame({'Mean': gd_mean,
    'Median': gd_median,
    'Variance': gd_variance,
    'Standard Deviation': gd_std,
    'SEM': gd_sem})


# disply dataframe
ss_df

In [ ]:
# A more advanced method to generate a summary statistics table of mean, median, variance, standard deviation,
# and SEM of the tumor volume for each regimen (only one method is required in the solution)

# Using the aggregation method, produce the same summary statistics in a single line(summary stats aggregated= ss_dfa )
ss_dfa = combined_data_clean[["Drug Regimen","Tumor Volume (mm3)"]].groupby(["Drug Regimen"]).agg(['mean', 'median','var', 'std', 'sem'])

# Display the summary statistics table
ss_dfa

## Bar and Pie Charts

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using Pandas.

pandas_DR = combined_data_clean.groupby('Drug Regimen')['Mouse ID'].count()

# Sort the data by count
pandas_DR_sorted = gd_DR.sort_values()

# Create a bar plot
pandas_DR_sorted.plot(kind='bar')

# Set the axis labels
plt.xlabel('Drug Regimen')
plt.ylabel('# of Observed Mouse Timepoints')

# Show the plot
plt.show()

In [ ]:
# Generate a bar plot showing the total number of rows (Mouse ID/Timepoints) for each drug regimen using pyplot.
pyplot_DR = combined_data_clean.groupby('Drug Regimen')['Mouse ID'].count()

# Sort the data by count
gd_DR_sorted = gd_DR.sort_values()

# Create a horizontal bar chart
plt.bar(gd_DR_sorted.index, gd_DR_sorted.values)

# Set the axis labels
plt.xlabel('Drug Regimen')
plt.xticks(rotation=90)
plt.ylabel('# of Observed Mouse Timepoints')

# Show the plot
plt.show()


In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using Pandas
cdc_df = combined_data_clean
# Identify distribution of data by sex
pandas_pc = cdc_df.groupby('Sex')['Mouse ID'].nunique()

# Plot a pie chart filled with corresponding percentages 
pandas_pc.plot = pandas_pc.plot(kind='pie', y='Sex', title='Proportion of Mice by Sex', autopct='%1.1f%%',startangle=180)

# Show the plot
plt.show()

In [ ]:
# Generate a pie plot showing the distribution of female versus male mice using pyplot

# Count the number of males and females
male_count = len(cdc_df[cdc_df['Sex'] == 'male'])
female_count = len(cdc_df[cdc_df['Sex'] == 'female'])


# Create a list of values to be plotted
values = [male_count, female_count]

# Create a list of labels for the pie chart
labels = ['Male', 'Female']

# Set the explode parameter to separate the slices of the pie chart
explode = [0.1, 0]

# Create the pie chart
plt.pie(values, labels=labels, explode=explode, autopct='%1.1f%%', startangle=180)

# Add a title to the pie chart
plt.title('Distribution by Sex')

# Show the pie chart
plt.show()

In [ ]:
cdc_df.isnull().values.any()

## Quartiles, Outliers and Boxplots

In [ ]:
# Calculate the final tumor volume of each mouse across four of the treatment regimens:  
# Capomulin, Ramicane, Infubinol, and Ceftamin

# Start by getting the last (greatest) timepoint for each mouse


# Merge this group df with the original DataFrame to get the tumor volume at the last timepoint


In [ ]:
# Put treatments into a list for for loop (and later for plot labels)


# Create empty list to fill with tumor vol data (for plotting)


# Calculate the IQR and quantitatively determine if there are any potential outliers. 

    
    # Locate the rows which contain mice on each drug and get the tumor volumes

    
    # add subset 

    
    # Determine outliers using upper and lower bounds


In [ ]:
# Generate a box plot that shows the distrubution of the tumor volume for each treatment group.


## Line and Scatter Plots

In [ ]:
# Generate a line plot of tumor volume vs. time point for a single mouse treated with Capomulin


In [ ]:
# Generate a scatter plot of mouse weight vs. the average observed tumor volume for the entire Capomulin regimen


## Correlation and Regression

In [ ]:
# Calculate the correlation coefficient and a linear regression model 
# for mouse weight and average observed tumor volume for the entire Capomulin regimen
